# **Set-up**

In [12]:
install.packages("xtable")
install.packages("hdm")
install.packages("randomForest")
install.packages("glmnet")
install.packages("sandwich")
install.packages("nnet")

Warning message:
"package 'xtable' is in use and will not be installed"
Warning message:
"package 'hdm' is in use and will not be installed"
Warning message:
"package 'randomForest' is in use and will not be installed"
Warning message:
"package 'glmnet' is in use and will not be installed"
Warning message:
"package 'sandwich' is in use and will not be installed"
Warning message:
"package 'nnet' is in use and will not be installed"


In [13]:
library(xtable)
library(randomForest)
library(hdm)
library(glmnet)
library(sandwich)
library(nnet)

# **I. Cleaning and Set-Up**

In [4]:
set.seed(1)

In [10]:
# Data
file <- "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/penn_jae.dat"
data <- read.table(file, header = TRUE)

# Mantener solo tg = 0 o 4
data <- data[data$tg %in% c(0, 4), ]

# Variable de tratamiento
data$T4 <- ifelse(data$tg == 4, 1, 0)

# Variable de resultado
data$log_inuidur1 <- log(data$inuidur1)

# Dummies de dep
data$dep_0 <- ifelse(data$dep == 0, 1, 0)
data$dep_1 <- ifelse(data$dep == 1, 1, 0)
data$dep_2 <- ifelse(data$dep == 2, 1, 0)

# Definir y, d, x
y <- as.matrix(data$log_inuidur1)
d <- as.matrix(data$T4)
x <- as.matrix(data[, c('female', 'black', 'othrace',
                         'dep_1', 'dep_2',
                         'q2', 'q3', 'q4', 'q5', 'q6',
                         'recall', 'agelt35', 'agegt54',
                         'durable', 'nondurable', 'lusd', 'husd')])
data

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,⋯,durable,nondurable,lusd,husd,muld,T4,log_inuidur1,dep_0,dep_1,dep_2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10824,0,18,18,0,0,0,0,2,0,⋯,0,0,0,1,0,0,2.890372,0,0,1
4,10824,0,1,1,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0.000000,1,0,0
5,10747,0,27,27,0,0,0,0,0,0,⋯,0,0,1,0,0,0,3.295837,1,0,0
12,10607,4,9,9,0,0,0,0,0,0,⋯,0,0,0,0,1,1,2.197225,1,0,0
13,10831,0,27,27,0,0,0,0,1,0,⋯,1,0,1,0,0,0,3.295837,0,1,0
14,10845,0,27,27,1,0,0,0,0,0,⋯,0,0,1,0,0,0,3.295837,1,0,0
15,10831,0,9,9,1,0,0,0,1,0,⋯,0,0,1,0,0,0,2.197225,0,1,0
17,10859,0,27,27,1,0,0,0,1,0,⋯,0,0,1,0,0,0,3.295837,0,1,0
23,10516,0,15,15,1,0,0,0,0,0,⋯,0,0,1,0,0,0,2.708050,1,0,0


# **II. Debiased ML**

In [17]:
# Función DML
DML2.for.PLM <- function(x, d, y, dreg, yreg, nfold=2) {
  nobs <- nrow(x)
  foldid <- rep.int(1:nfold, times = ceiling(nobs/nfold))[sample.int(nobs)]
  I <- split(1:nobs, foldid)
  ytil <- dtil <- rep(NA, nobs)
  cat("fold: ")
  for(b in 1:length(I)){
    dfit <- dreg(x[-I[[b]],], d[-I[[b]]])
    yfit <- yreg(x[-I[[b]],], y[-I[[b]]])
    dhat <- predict(dfit, x[I[[b]],], type="response")
    yhat <- predict(yfit, x[I[[b]],], type="response")
    dtil[I[[b]]] <- (d[I[[b]]] - dhat)
    ytil[I[[b]]] <- (y[I[[b]]] - yhat)
    cat(b," ")
  }
  rfit <- lm(ytil ~ dtil)
  coef.est <- coef(rfit)[2]
  se <- sqrt(vcovHC(rfit)[2,2])
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est, se))
  return(list(coef.est=coef.est, se=se, dtil=dtil, ytil=ytil))
}

# DML con OLS
cat(sprintf("\nDML con OLS\n"))
dreg <- function(x,d){ glmnet(x, d, lambda = 0) }
yreg <- function(x,y){ glmnet(x, y, lambda = 0) }
DML2.OLS = DML2.for.PLM(x, d, y, dreg, yreg, nfold=10)

# DML con Lasso
cat(sprintf("\nDML con Lasso\n"))
dreg <- function(x,d){ rlasso(x, d, post=FALSE) }
yreg <- function(x,y){ rlasso(x, y, post=FALSE) }
DML2.lasso = DML2.for.PLM(x, d, y, dreg, yreg, nfold=10)

# DML con Random Forest
cat(sprintf("\nDML con Random Forest\n"))
dreg <- function(x,d){ suppressWarnings(randomForest(x, d)) }
yreg <- function(x,y){ suppressWarnings(randomForest(x, y)) }
DML2.RF = DML2.for.PLM(x, d, y, dreg, yreg, nfold=10)

# DML con Neural Network
DML2.for.PLM.NN <- function(x, d, y, dreg, yreg, nfold=2) {
  nobs <- nrow(x)
  foldid <- rep.int(1:nfold, times = ceiling(nobs/nfold))[sample.int(nobs)]
  I <- split(1:nobs, foldid)
  ytil <- dtil <- rep(NA, nobs)
  cat("fold: ")
  for(b in 1:length(I)){
    dfit <- dreg(x[-I[[b]],], d[-I[[b]]])
    yfit <- yreg(x[-I[[b]],], y[-I[[b]]])
    dhat <- predict(dfit, x[I[[b]],])
    yhat <- predict(yfit, x[I[[b]],])
    dtil[I[[b]]] <- (d[I[[b]]] - dhat)
    ytil[I[[b]]] <- (y[I[[b]]] - yhat)
    cat(b," ")
  }
  rfit <- lm(ytil ~ dtil)
  coef.est <- coef(rfit)[2]
  se <- sqrt(vcovHC(rfit)[2,2])
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est, se))
  return(list(coef.est=coef.est, se=se, dtil=dtil, ytil=ytil))
}

cat(sprintf("\nDML con Neural Network\n"))
dreg <- function(x,d){ nnet(x, d, size=5, linout=TRUE, trace=FALSE, maxit=1000) }
yreg <- function(x,y){ nnet(x, y, size=5, linout=TRUE, trace=FALSE, maxit=1000) }
DML2.NN = DML2.for.PLM.NN(x, d, y, dreg, yreg, nfold=10)

# RMSE y Tabla
prRes.D <- c(mean((DML2.OLS$dtil)^2), mean((DML2.lasso$dtil)^2), mean((DML2.RF$dtil)^2), mean((DML2.NN$dtil)^2))
prRes.Y <- c(mean((DML2.OLS$ytil)^2), mean((DML2.lasso$ytil)^2), mean((DML2.RF$ytil)^2), mean((DML2.NN$ytil)^2))
prRes <- rbind(sqrt(prRes.D), sqrt(prRes.Y))


DML con OLS
fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = -0.0725686 (0.0351348)

DML con Lasso
fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = -0.0796961 (0.0353666)

DML con Random Forest
fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = -0.076369 (0.0349711)

DML con Neural Network
fold: 1  2  3  4  5  6  7  8  9  10  
coef (se) = -0.101712 (0.0472255)


In [18]:
# Tabla de resultados
table <- matrix(0, 4, 4)
table[1,1] <- as.numeric(DML2.OLS$coef.est)
table[2,1] <- as.numeric(DML2.lasso$coef.est)
table[3,1] <- as.numeric(DML2.RF$coef.est)
table[4,1] <- as.numeric(DML2.NN$coef.est)

table[1,2] <- as.numeric(DML2.OLS$se)
table[2,2] <- as.numeric(DML2.lasso$se)
table[3,2] <- as.numeric(DML2.RF$se)
table[4,2] <- as.numeric(DML2.NN$se)

table[1,3] <- as.numeric(prRes[2,1])
table[2,3] <- as.numeric(prRes[2,2])
table[3,3] <- as.numeric(prRes[2,3])
table[4,3] <- as.numeric(prRes[2,4])

table[1,4] <- as.numeric(prRes[1,1])
table[2,4] <- as.numeric(prRes[1,2])
table[3,4] <- as.numeric(prRes[1,3])
table[4,4] <- as.numeric(prRes[1,4])

colnames(table) <- c("Estimate","Standard Error", "RMSE Y", "RMSE D")
rownames(table) <- c("OLS", "Lasso", "RF", "Neural Network")
table

,Estimate,Standard Error,RMSE Y,RMSE D
OLS,-0.07256863,0.03513480,1.194740,0.4751677
Lasso,-0.07969611,0.03536656,1.198766,0.4743213
RF,-0.07636905,0.03497112,1.206104,0.4808657
Neural Network,-0.10171178,0.04722554,1.437708,0.4859913


In [19]:
print(table, digit=3)

               Estimate Standard Error RMSE Y RMSE D
OLS             -0.0726         0.0351   1.19  0.475
Lasso           -0.0797         0.0354   1.20  0.474
RF              -0.0764         0.0350   1.21  0.481
Neural Network  -0.1017         0.0472   1.44  0.486


## Modelo

Basándome en los resultados, elijo **OLS** y **Lasso** como los modelos más adecuados.  
**OLS** tiene el RMSE más bajo para predecir *D* (0.475) y un valor competitivo para *Y* (1.19).  
**Lasso** obtiene resultados muy parecidos, con el RMSE D más bajo (0.474) y apenas un poco más alto en *RMSE Y* (1.20).  
Además, ambos tienen errores estándar pequeños (~0.035), lo que indica estimaciones bastante precisas.

El **Random Forest** muestra RMSE un poco más altos. Además, el **Neural Network** es claramente el peor modelo, pues su *RMSE Y* es de 1.44 (mucho mayor que los demás) y su error estándar es 0.047, casi un 35% más alto que los otros métodos.

Por lo tanto, utilizo **OLS y Lasso** para estimar el efecto causal.  
Los resultados son muy similares:  
- **OLS:** coeficiente = -0.0726 (se = 0.0351)  
- **Lasso:** coeficiente = -0.0797 (se = 0.0354)

Esto indica que el tratamiento reduce el logaritmo de la duración del desempleo en aproximadamente 0.07–0.08 unidades, un efecto negativo y estadísticamente significativo, consistente entre ambos métodos.

# **III. No cross-fitting**

In [20]:
DML.no.crossfit <- function(x, d, y, dreg, yreg) {
  dfit <- dreg(x, d)
  yfit <- yreg(x, y)
  dhat <- predict(dfit, x, type="response")
  yhat <- predict(yfit, x, type="response")
  dtil <- d - dhat
  ytil <- y - yhat
  rfit <- lm(ytil ~ dtil)
  coef.est <- coef(rfit)[2]
  se <- sqrt(vcovHC(rfit)[2,2])
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est, se))
  return(list(coef.est=coef.est, se=se, dtil=dtil, ytil=ytil))
}

# Función para Neural Network (sin type="response")
DML.no.crossfit.NN <- function(x, d, y, dreg, yreg) {
  dfit <- dreg(x, d)
  yfit <- yreg(x, y)
  dhat <- predict(dfit, x)
  yhat <- predict(yfit, x)
  dtil <- d - dhat
  ytil <- y - yhat
  rfit <- lm(ytil ~ dtil)
  coef.est <- coef(rfit)[2]
  se <- sqrt(vcovHC(rfit)[2,2])
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est, se))
  return(list(coef.est=coef.est, se=se, dtil=dtil, ytil=ytil))
}

# OLS
cat(sprintf("\nDML sin cross-fitting: OLS\n"))
dreg <- function(x,d){ glmnet(x, d, lambda = 0) }
yreg <- function(x,y){ glmnet(x, y, lambda = 0) }
DML.OLS.nocv = DML.no.crossfit(x, d, y, dreg, yreg)

# Lasso
cat(sprintf("\nDML sin cross-fitting: Lasso\n"))
dreg <- function(x,d){ rlasso(x, d, post=FALSE) }
yreg <- function(x,y){ rlasso(x, y, post=FALSE) }
DML.lasso.nocv = DML.no.crossfit(x, d, y, dreg, yreg)

# Random Forest
cat(sprintf("\nDML sin cross-fitting: Random Forest\n"))
dreg <- function(x,d){ suppressWarnings(randomForest(x, d)) }
yreg <- function(x,y){ suppressWarnings(randomForest(x, y)) }
DML.RF.nocv = DML.no.crossfit(x, d, y, dreg, yreg)

# Neural Network
cat(sprintf("\nDML sin cross-fitting: Neural Network\n"))
dreg <- function(x,d){ nnet(x, d, size=5, linout=TRUE, trace=FALSE, maxit=1000) }
yreg <- function(x,y){ nnet(x, y, size=5, linout=TRUE, trace=FALSE, maxit=1000) }
DML.NN.nocv = DML.no.crossfit.NN(x, d, y, dreg, yreg)

# RMSE
prRes.D.nocv <- c(mean((DML.OLS.nocv$dtil)^2), mean((DML.lasso.nocv$dtil)^2), 
                   mean((DML.RF.nocv$dtil)^2), mean((DML.NN.nocv$dtil)^2))
prRes.Y.nocv <- c(mean((DML.OLS.nocv$ytil)^2), mean((DML.lasso.nocv$ytil)^2), 
                   mean((DML.RF.nocv$ytil)^2), mean((DML.NN.nocv$ytil)^2))
prRes.nocv <- rbind(sqrt(prRes.D.nocv), sqrt(prRes.Y.nocv))


DML sin cross-fitting: OLS

coef (se) = -0.0725603 (0.0351333)

DML sin cross-fitting: Lasso

coef (se) = -0.0791692 (0.0352883)

DML sin cross-fitting: Random Forest

coef (se) = -0.0759841 (0.035442)

DML sin cross-fitting: Neural Network

coef (se) = -0.0841915 (0.0361247)


In [21]:
# Tabla
table.nocv <- matrix(0, 4, 4)
table.nocv[1,1] <- as.numeric(DML.OLS.nocv$coef.est)
table.nocv[2,1] <- as.numeric(DML.lasso.nocv$coef.est)
table.nocv[3,1] <- as.numeric(DML.RF.nocv$coef.est)
table.nocv[4,1] <- as.numeric(DML.NN.nocv$coef.est)

table.nocv[1,2] <- as.numeric(DML.OLS.nocv$se)
table.nocv[2,2] <- as.numeric(DML.lasso.nocv$se)
table.nocv[3,2] <- as.numeric(DML.RF.nocv$se)
table.nocv[4,2] <- as.numeric(DML.NN.nocv$se)

table.nocv[1,3] <- as.numeric(prRes.nocv[2,1])
table.nocv[2,3] <- as.numeric(prRes.nocv[2,2])
table.nocv[3,3] <- as.numeric(prRes.nocv[2,3])
table.nocv[4,3] <- as.numeric(prRes.nocv[2,4])

table.nocv[1,4] <- as.numeric(prRes.nocv[1,1])
table.nocv[2,4] <- as.numeric(prRes.nocv[1,2])
table.nocv[3,4] <- as.numeric(prRes.nocv[1,3])
table.nocv[4,4] <- as.numeric(prRes.nocv[1,4])

colnames(table.nocv) <- c("Estimate","Standard Error", "RMSE Y", "RMSE D")
rownames(table.nocv) <- c("OLS", "Lasso", "RF", "Neural Network")

print(table.nocv, digits=3)

               Estimate Standard Error RMSE Y RMSE D
OLS             -0.0726         0.0351   1.19  0.473
Lasso           -0.0792         0.0353   1.20  0.474
RF              -0.0760         0.0354   1.12  0.445
Neural Network  -0.0842         0.0361   1.21  0.469


## Modelo (Sin Cross-Fitting)

Basándome en los resultados sin cross-fitting, elijo **Random Forest** como el modelo más adecuado.  
**Random Forest** presenta el RMSE más bajo tanto para predecir *Y* (1.12) como para *D* (0.445), superando a los demás métodos.  
Su error estándar (0.0354) es muy similar al de **OLS** y **Lasso**, lo que indica una estimación precisa y estable.

**OLS** y **Lasso** también muestran buenos resultados, con valores de *RMSE Y* entre 1.19 y 1.20, y *RMSE D* alrededor de 0.47, pero su desempeño es ligeramente inferior al de **Random Forest**.  
El **Neural Network** tiene el *RMSE Y* más alto (1.21) y un error estándar un poco mayor (0.0361), aunque su *RMSE D* (0.469) es competitivo.

Por lo tanto, utilizo **Random Forest** para estimar el efecto causal sin cross-fitting.  
El resultado obtenido es:  
- **Random Forest:** coeficiente = -0.0760 (se = 0.0354)

Esto indica que el tratamiento reduce el logaritmo de la duración del desempleo en aproximadamente 0.076 unidades, un efecto negativo y estadísticamente significativo.

## Comparación: Con vs. Sin Cross-Fitting

### 1. ¿Qué puedo decir sobre el RMSE para predecir *y* y *d*?
Los RMSE sin cross-fitting son más bajos, sobre todo en los modelos más flexibles.  
Por ejemplo:
- **Random Forest:** el RMSE de *Y* baja de **1.21** a **1.12** y el de *D* de **0.481** a **0.445**.  
- **Neural Network:** el RMSE de *Y* disminuye de **1.44** a **1.21**, y el de *D* de **0.486** a **0.469**.  
- En **OLS** y **Lasso**, los cambios son mínimos, prácticamente iguales entre ambos métodos.

En general, los modelos más complejos (RF y NN) muestran las reducciones más grandes, mientras que los lineales se mantienen estables.

### 2. ¿Por qué una función da RMSE menor que la otra?
Los RMSE son menores sin cross-fitting porque hay **sobreajuste (overfitting)**.  
En ese caso, el modelo se entrena y se evalúa sobre los mismos datos, por lo que termina “aprendiéndose” el ruido del conjunto de entrenamiento.  
Esto hace que los errores parezcan más pequeños, pero en realidad el modelo no está generalizando bien.

Los métodos más flexibles, como **Random Forest** y **Neural Network**, tienen más capacidad para ajustarse a los datos, por eso muestran las mayores caídas en RMSE.  
Con **cross-fitting**, los modelos predicen sobre datos que no vieron durante el entrenamiento, así que los RMSE son más altos, pero **más realistas y confiables**.

### 3. ¿Qué problema hay si estimamos sin cross-fitting?
El problema principal es que **las estimaciones del efecto causal se sesgan**.  
Sin cross-fitting, los residuales están correlacionados con las predicciones porque se usan los mismos datos para entrenar y estimar.  
Esto hace que el coeficiente del tratamiento no sea totalmente confiable y que los errores estándar estén mal calculados.

En cambio, **el cross-fitting evita ese sesgo** al usar muestras distintas para entrenar y para predecir, garantizando que los residuales sean independientes.  
Así, los resultados son más válidos para inferencia causal y podemos usar modelos flexibles sin perder rigor estadístico.